In [1]:
import argparse
import logging

import yaml

parser = argparse.ArgumentParser(description="GAT")
parser.add_argument(
    "--task",
    type=str,
    default="classification",
    choices=["classification", "clustering"],
)
parser.add_argument("--dataset", type=str, default="acm")
args, _ = parser.parse_known_args()


def load_best_configs(args, path):
    with open(path, "r") as f:
        configs = yaml.load(f, yaml.FullLoader)

    if args.dataset not in configs:
        logging.info("Best args not found")
        return args

    configs = configs[args.dataset]

    for k, v in configs.items():
        if "lr" in k or "weight_decay" in k:
            v = float(v)
        setattr(args, k, v)
    print("------ Use best configs ------")
    return args

In [13]:
parser = argparse.ArgumentParser(description="GAT")
parser.add_argument(
    "--task",
    type=str,
    default="classification",
    choices=["classification", "clustering"],
)
parser.add_argument("--dataset", type=str, default="acm")
args, _ = parser.parse_known_args()

args = load_best_configs(args, "./configs.yml")

------ Use best configs ------


In [15]:
acm.meta_paths_dict

{'PAP': [('paper', 'paper-author', 'author'),
  ('author', 'author-paper', 'paper')],
 'PSP': [('paper', 'paper-subject', 'subject'),
  ('subject', 'subject-paper', 'paper')]}

In [3]:
from openhgnn.dataset.NodeClassificationDataset import OHGB_NodeClassification

E:\Anacodna3\envs\GML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
acm = OHGB_NodeClassification("ohgbn-acm", raw_dir="./dataset", logger=None)

Extracting file to ./openhgnn/dataset\ohgbn-acm
Done saving data into cached files.


In [7]:
hg = acm.g
metapaths_dict = acm.meta_paths_dict

In [8]:
setattr(args,'use_mp2vec_feat_pred','false')

In [17]:
import importlib

import dgl
import hgmae
import torch

importlib.reload(hgmae)
from hgmae import HGMAE

In [18]:
h_dict=hg.ndata['h']
mp2vec_feat_dict=hg.ndata['pap_m2v_emb'].copy()
mp2vec_feat_dict
for k,v in  hg.ndata['psp_m2v_emb'].items():
    mp2vec_feat_dict[k]=torch.concat([mp2vec_feat_dict[k],v],dim=1)

In [19]:
model = HGMAE.build_model_from_args(args, metapaths_dict)

Namespace(task='classification', dataset='acm', category='paper', activation='prelu', attr_restore_gamma=1, attr_restore_loss_weight=1, decoder='han', encoder='han', eva_lr=0.01, eva_wd=0.0005, feat_drop=0.2, attn_drop=0.5, node_mask_rate='0.5,0.005,0.8', attr_unchanged_rate=0.3, attr_replace_rate=0.2, gpu=0, in_dim=1902, hidden_dim=256, l2_coef=0, loss_func='sce', lr=0.0008, mae_epochs=3, mask_rate=0.4, mp2vec_feat_dim=128, mp2vec_feat_gamma=2, mp2vec_feat_drop=0.2, mp2vec_feat_pred_loss_weight=0.1, mp_edge_gamma=3, mp_edge_mask_rate=0.7, mp_edge_recon_loss_weight=1, mps_batch_size=256, mps_context_size=3, mps_epoch=1, mps_lr=0.001, mps_num_negative_samples=3, mps_walk_length=10, mps_walks_per_node=3, n_labels=3, negative_slope=0.2, nei_num=2, norm='batchnorm', num_heads=4, num_layers=2, num_out_heads=1, optimizer='adam', patience=10, residual=False, scheduler=True, scheduler_gamma=0.999, use_mp2vec_feat_pred=True, use_mp_edge_recon=True)
<class 'argparse.Namespace'>


In [12]:
model.forward(hg,h_dict,mp2vec_feat_dict,epoch=3)

tensor(0.1608)
tensor(1.0022)
tensor(1.0420)


tensor(1.2672, grad_fn=<AddBackward0>)

In [25]:
import torch.nn as nn


In [20]:
from openhgnn.models import HAN

In [21]:
HAN.build_model_from_args()

openhgnn.models.HAN.HAN

In [41]:
ntypes=set()
ntypes.add('paper')
ntype_meta_paths_dict = {}
for ntype in ntypes:
    ntype_meta_paths_dict[ntype] = {}
    for meta_path_name, meta_path in metapaths_dict.items():
        # a meta path starts with this node type
        if meta_path[0][0] == ntype:
            ntype_meta_paths_dict[ntype][meta_path_name] = meta_path

In [43]:
ntype_meta_paths_dict

{'paper': {'PAP': [('paper', 'paper-author', 'author'),
   ('author', 'author-paper', 'paper')],
  'PSP': [('paper', 'paper-subject', 'subject'),
   ('subject', 'subject-paper', 'paper')]}}

In [54]:
in_dim=1902
hidden_dim=256
out_dim=256

In [58]:
han=HAN(ntype_meta_paths_dict,in_dim,hidden_dim,out_dim,[4,4],dropout=0.5)


In [59]:
out=han.forward(hg,hg.ndata['h'])

In [61]:
out['paper']

tensor([[-0.0050, -0.0119,  0.0775,  ...,  0.1484, -0.0008, -0.2171],
        [-0.0512, -0.0030, -0.0563,  ...,  0.1167, -0.0183, -0.0856],
        [-0.0559, -0.0975, -0.0848,  ...,  0.0208,  0.0353, -0.0796],
        ...,
        [-0.0968,  0.0197,  0.0212,  ...,  0.0378, -0.1055, -0.1342],
        [-0.0207,  0.0193, -0.1038,  ..., -0.2370, -0.0806, -0.1783],
        [ 0.0111,  0.0468,  0.0575,  ..., -0.0594,  0.0437, -0.0510]],
       grad_fn=<AddmmBackward0>)

In [69]:
from dgl.nn import GATConv

g = dgl.graph(([0,1,2,3,2,5], [1,2,3,4,0,3]))
g = dgl.add_self_loop(g)
feat = torch.ones(6, 10)
gatconv = GATConv(10, 2, num_heads=3)
res = gatconv(g, feat)

In [73]:
res.flatten(1)

tensor([[ 1.9662,  0.9604, -2.1907,  1.9467, -0.7873,  0.7456],
        [ 1.9662,  0.9604, -2.1907,  1.9467, -0.7873,  0.7456],
        [ 1.9662,  0.9604, -2.1907,  1.9467, -0.7873,  0.7456],
        [ 1.9662,  0.9604, -2.1907,  1.9467, -0.7873,  0.7456],
        [ 1.9662,  0.9604, -2.1907,  1.9467, -0.7873,  0.7456],
        [ 1.9662,  0.9604, -2.1907,  1.9467, -0.7873,  0.7456]],
       grad_fn=<ViewBackward0>)